# Python sockets 101 - Server side
- ## Sébastien Tixeuil
- ## LINCS Python Workshop 14 December 2020

# Source
- <https://www.apress.com/gp/book/9781430258544>

![](https://media.springernature.com/w153/springer-static/cover/book/9781430258551.jpg)

# UDP Server (Python 2X)

In [ ]:
from socket import *
serverPort = 1235
serverSocket = socket(AF_INET,SOCK_DGRAM)
serverSocket.bind(('',serverPort))
print('server ready')
while True:
    message, clientAddress = serverSocket.recvfrom(2048)
    modifiedMessage = message.upper()
    serverSocket.sendto(modifiedMessage,clientAddress)

# UDP Server (Python 3X)

In [ ]:
from socket import *
serverPort = 1235
serverSocket = socket(AF_INET,SOCK_DGRAM)
serverSocket.bind(('',serverPort))
print('server ready')
while True:
    message, clientAddress = serverSocket.recvfrom(2048)
    modifiedMessage = message.decode('utf-8').upper()
    print(modifiedMessage)
    serverSocket.sendto(modifiedMessage.encode('utf-8'),clientAddress)

# UDP Server (TTL Estimator)

In [ ]:
from socket import *

serverPort = 1234
serverSocket = socket(AF_INET,SOCK_DGRAM)

serverSocket.bind(('',serverPort))

print('server ready')

while True:
    _, clientAddress = serverSocket.recvfrom(2048)
    serverSocket.sendto(b'PONG',clientAddress)

# TCP Server (Sequential)

In [ ]:
from socket import *
serverPort = 1235
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.bind(('',serverPort))
serverSocket.listen(1)
print('server ready')
while True:
    connectionSocket, address = serverSocket.accept()
    message = connectionSocket.recv(4096)
    modifiedMessage = message.decode('utf-8').upper()
    connectionSocket.send(modifiedMessage.encode('‘utf-8'))
    connectionSocket.close()

# IPv4 Names

In [ ]:
from socket import *

print(gethostname())

print(getfqdn())

print(gethostbyname('lip6.fr'))

print(gethostbyaddr('132.227.104.15'))

print(gethostbyname(getfqdn()))

# IPv4/IPv6 Names

In [ ]:
from socket import *

infolist = getaddrinfo('lip6.fr','www')

print(infolist)

info = infolist[1]

print(info)

s = socket(*info[0:3])

s.connect(info[4])

s.close()

infolist = getaddrinfo('google.fr','www')
print(infolist)

infolist = getaddrinfo('lincs.fr','www')
print(infolist)

# Byte Order over the Network

In [ ]:
from struct import *

print(hex(1234))

print(pack('<i',1234))
print(pack('>i',1234))
print(pack('!i',1234))

print(unpack('>i',b'\x00\x00\x04\xd2'))

print(unpack('!i',b'\x00\x00\x04\xd2'))

# Network Exceptions

- `OSError`: almost every failure that can happen during a network connection
- `socket.gaierror`: address-related error
- `socket.timeout`: timeout expired

In [ ]:
from socket import *

try:
    infolist = getaddrinfo('nonexistent.com','www')
except gaierror:
    print('This host does not seem to exist')

# TCP Server with `recvall`

In [ ]:
from struct import *
from socket import *

header_struct = Struct('!I')

def recvall(sock, length):
    blocks = []
    while length:
        block = sock.recv(length)
        if not block:
            raise EOFError('socket closed with %d bytes left'
                           ' in this block'.format(length))
        length -= len(block)
        blocks.append(block)
    return b''.join(blocks)

def put_block(sock, message):
    block_length = len(message)
    sock.sendall(header_struct.pack(block_length))
    sock.sendall(message)
    
def get_block(sock):
    data = recvall(sock, header_struct.size)
    (block_length,) = header_struct.unpack(data)
    return recvall(sock, block_length)

from socket import *
serverPort = 1241
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.bind(('',serverPort))
serverSocket.listen(1)
print('server ready')
while True:
    connectionSocket, address = serverSocket.accept()
    message = get_block(connectionSocket)
    modifiedMessage = message.decode('utf-8').upper()
    put_block(connectionSocket,modifiedMessage.encode('‘utf-8'))
    connectionSocket.close()

# Sequential TCP Server (with Function)

In [ ]:
from socket import *
serverPort = 1234
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.bind(('',serverPort))
serverSocket.listen(1)
print('server ready')

def handle_client(clientSocket):
    try:
        while True:
            received = clientSocket.recv(4096)
            if not received:
                clientSocket.close()
            else:
                to_send = received.decode('utf-8').upper().encode('utf-8')
                clientSocket.sendall(to_send)
    except OSError:
        clientSocket.close()
        
while True:
        connectionSocket, address = serverSocket.accept()
        handle_client(connectionSocket)


# The impact of `listen`

In [ ]:
from socket import *
serverPort = 1236
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.bind(('',serverPort))
serverSocket.listen(100)
print('server ready')

def handle_client(clientSocket):
    try:
        while True:
            received = clientSocket.recv(4096)
            if not received:
                clientSocket.close()
            else:
                to_send = received.decode('utf-8').upper().encode('utf-8')
                clientSocket.sendall(to_send)
    except OSError:
        clientSocket.close()
        
while True:
        connectionSocket, address = serverSocket.accept()
        handle_client(connectionSocket)

# (Slow) Sequential TCP Server

In [ ]:
from socket import *
from time import *

serverPort = 1237
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.bind(('',serverPort))
serverSocket.listen(100)
print('server ready')

def handle_client(clientSocket):
    try:
        while True:
            received = clientSocket.recv(4096)
            if not received:
                clientSocket.close()
            else:
                sleep(5)
                to_send = received.decode('utf-8').upper().encode('utf-8')
                clientSocket.sendall(to_send)
    except OSError:
        clientSocket.close()
        
while True:
        connectionSocket, address = serverSocket.accept()
        handle_client(connectionSocket)


# Multiplexing TCP server

In [ ]:
from select import *
from socket import *

serverPort = 1236
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.bind(('',serverPort))
serverSocket.listen(100)
print('server ready')

my_poll = poll()
my_poll.register(serverSocket,POLLIN)

sockets = {serverSocket.fileno(): serverSocket} 
# retrieve socket object from fileno
received = dict()  
# bytes received from fileno, that are not yet processed
to_send = dict() 
# bytes to be sent from fileno, that have been processed

while True:
    for fd, event in my_poll.poll():
        if event & (POLLHUP|POLLERR|POLLNVAL):
            if fd in received:
                received.pop(fd)
            if fd in to_send:
                to_send.pop(fd)
            my_poll.unregister(fd)
            del sockets[fd]
            if fd in sockets:
                sockets.pop(fd)
        elif sockets[fd] is serverSocket:
            connectionSocket, address = serverSocket.accept()
            sockets[connectionSocket.fileno()] = connectionSocket
            my_poll.register(connectionSocket, POLLIN)
        else:
            if event & POLLIN:
                data = sockets[fd].recv(4096)
                if not data:
                    sockets[fd].close()
                    continue
                if fd in received:
                    received[fd] += data
                else:
                    received[fd] = data
                my_poll.modify(fd,POLLIN|POLLOUT)
            if event & POLLOUT:
                data = received.pop(fd).decode('utf-8')
                data = data.upper().encode('utf-8')
                if fd in to_send:
                    data = to_send.pop(fd) + data
                n = sockets[fd].send(data)
                if n < len(data):
                    to_send[fd] = data[n:]
                else:
                    my_poll.modify(fd,POLLIN)
                                    

# Multithread TCP Server (one thread per client)

In [ ]:
from socket import *
from time import *
from threading import *

serverPort = 1237
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.bind(('',serverPort))
serverSocket.listen(100)
print('server ready')

def handle_client(clientSocket):
    try:
        while True:
            received = clientSocket.recv(4096)
            if not received:
                clientSocket.close()
            else:
                to_send = received.decode('utf-8').upper().encode('utf-8')
                clientSocket.sendall(to_send)
    except OSError:
        clientSocket.close()
        
while True:
    connectionSocket, address = serverSocket.accept()
    Thread(target=handle_client,args=(connectionSocket,)).start()

# Multithread TCP Server (one slow thread per client)

In [ ]:
from socket import *
from time import *
from threading import *

serverPort = 1238
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.bind(('',serverPort))
serverSocket.listen(100)
print('server ready')

def handle_client(clientSocket):
    try:
        while True:
            received = clientSocket.recv(4096)
            if not received:
                clientSocket.close()
            else:
                sleep(5)
                to_send = received.decode('utf-8').upper().encode('utf-8')
                clientSocket.sendall(to_send)
    except OSError:
        clientSocket.close()
        
while True:
    connectionSocket, address = serverSocket.accept()
    Thread(target=handle_client,args=(connectionSocket,)).start()

# Multithread TCP Server (pool of threads)

In [ ]:
from socket import *
from time import *
from threading import *

serverPort = 1239
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.bind(('',serverPort))
serverSocket.listen(100)
print('server ready')

def handle_client(listeningSocket):
    try:
        while True:
            clientSocket, address = listeningSocket.accept()
            received = clientSocket.recv(4096)
            if not received:
                clientSocket.close()
            else:
                to_send = received.decode('utf-8').upper().encode('utf-8')
                clientSocket.sendall(to_send)
    except OSError:
        clientSocket.close()
        
for i in range(4):
    Thread(target=handle_client,args=(serverSocket,)).start()

# Multi-thread TCP Server (Pool of Threads) with timeout 

In [ ]:
from socket import *
from time import *
from threading import *

serverPort = 1240
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.settimeout(20)

serverSocket.bind(('',serverPort))
serverSocket.listen(100)
print('server ready')

def handle_client(listeningSocket):
    try:
        while True:
            clientSocket, address = listeningSocket.accept()
            received = clientSocket.recv(4096)
            if not received:
                clientSocket.close()
            else:
                to_send = received.decode('utf-8').upper().encode('utf-8')
                clientSocket.sendall(to_send)
    except timeout:
        print("Socket timed out, exiting thread")
        clientSocket.close()
    except OSError:
        clientSocket.close()
        
for i in range(4):
    Thread(target=handle_client,args=(serverSocket,)).start()

# Multi-thread TCP Server (Pool of Slow Threads) with timeout

In [ ]:
from socket import *
from time import *
from threading import *

serverPort = 1241
serverSocket = socket(AF_INET,SOCK_STREAM)
serverSocket.setsockopt(SOL_SOCKET,SO_REUSEADDR,1)
serverSocket.settimeout(20)

serverSocket.bind(('',serverPort))
serverSocket.listen(100)
print('server ready')

def handle_client(listeningSocket):
    try:
        while True:
            clientSocket, address = listeningSocket.accept()
            received = clientSocket.recv(4096)
            if not received:
                clientSocket.close()
            else:
                sleep(5)
                to_send = received.decode('utf-8').upper().encode('utf-8')
                clientSocket.sendall(to_send)
    except timeout:
        print("Socket timed out, exiting thread")
        clientSocket.close()
    except OSError:
        clientSocket.close()
        
for i in range(4):
    Thread(target=handle_client,args=(serverSocket,)).start()